In [1]:
import raserManipulation as RM
import myServices as ms
import os
import rasterio as rst

In [4]:
## Testing reprojection
cwd ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
rasterTool = RM.generalRasterTools(cwd)

sourcName = 'dtm_1m_utm13_e_4_159.tif'
sourcePath = os.path.join(cwd,sourcName)

inName = 'SK_Regina_2019_2ForTest_5m.tif'
inputPath = os.path.join(cwd,inName)

outName = 'SK_Regina_2019_2ForTest_5m_CRS.tif'
outPath = os.path.join(cwd,outName)


crs, trns = rasterTool.get_CRSAndTranslation_GTIFF(sourcePath) 

res = rasterTool.get_rasterResolution(inputPath)
print ('original transformatin', trns)
newTrns = rasterTool.ensureTranslationResolution(trns,res) 
print ('newTrns transformatin', newTrns)

rasterTool.set_CRSAndTranslation_GTIF(inputPath, outPath, crs, newTrns)

# with rst.open(sourcePath) as src: 
#             profile = src.profile
#             src.close()


# print('crs, trns : ',newTrns)




Current working directory :  /Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir
original transformatin | 1.00, 0.00, 540000.00|
| 0.00,-1.00, 5600000.00|
| 0.00, 0.00, 1.00|
newTrns transformatin | 5.00, 0.00, 540000.00|
| 0.00,-5.00, 5600000.00|
| 0.00, 0.00, 1.00|
This is a profile : {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -32767.0, 'width': 8000, 'height': 4000, 'count': 1, 'crs': CRS.from_wkt('LOCAL_CS["unnamed",UNIT["unknown",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(5.0, 0.0, 510000.0,
       0.0, -5.0, 5600000.0), 'tiled': False, 'compress': 'deflate', 'interleave': 'band'}
#######  set_CRSAndTranslation_GTIF  ##########
original kwds : {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -32767.0, 'width': 8000, 'height': 4000, 'count': 1, 'crs': CRS.from_wkt('LOCAL_CS["unnamed",UNIT["unknown",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(5.0, 0.0, 510000.0,
       0.0, -5.0, 5600000.0

True

In [10]:
trans = newTrns['transform']
print(trans[0])
newTrans = rst.Affine(5,trans[1],trans[2],trans[3],-5,trans[5] )


1.0


In [ ]:
#### Print UTM zone by Ftp_dtm address in *.csv
cwd = '/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI'
file_list = ms.listFreeFilesInDirByExt(cwd, ext ='.csv')
for i in file_list:
    print(i) 
    path = os.path.join(cwd,i)
    with open(path,'r+') as file:
#         reader = csv.reader(file)
        for r in file:
            init = r.find("utm")
            if init > 0:
                end = init+5  
                print(r[init:end])


In [ ]:
### Read ALL *.CSV in cwd, download all DTM_htp by *.CSV, compute Mosaik and Resampling.

cwd ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/'

file_list = ms.listFreeFilesInDirByExt(cwd, ext ='.csv')
print(file_list)
rasterTool = RM.generalRasterTools(cwd)
for file in file_list:
    rasterTool.mosaikAndResamplingFromCSV(file,5,'Ftp_dtm',crearTransitDir = False)
    

'ON_NRCAN_PEEL.csv', 
'ON_NRCAN_York_2019.csv',
'NS_NSDNR_2020.csv', 
'ON_Lake_Erie_2018.csv',  
'NS_NSDNR_2019_2_2.csv', 
'NS_NSDNR_2018.csv', 

In [ ]:
# Excecute ONLY RESAMPLING from TransitDir
transitFolderPath ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
k = ms.listFreeFilesInDirByExt(transitFolderPath, ext = '.tif')
name,ext = ms.splitFilenameAndExtention(k[0])
rasterTool = RM.generalRasterTools(transitFolderPath)
resampledRaster = ms.makePath(transitFolderPath,(name + '_5m.tif'))
rasterTool.rasterResampler(k, resampledRaster, 5)

In [ ]:
## Download individuals files
dtmFtpList = ['https://ftp.maps.canada.ca/pub/elevation/dem_mne/highresolution_hauteresolution/dtm_mnt/1m/NRCAN/GTA2015/utm17/dtm_1m_utm17_e_10_82.tif']
transitFolderPath ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
RM.downloadTailsToLocalDir(dtmFtpList,transitFolderPath)

In [ ]:
# Excecute ONLY MOSAIK from TransitDir
transitFolderPath = '/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
dtmTail = ms.listFreeFilesInDirByExt(transitFolderPath, ext ='.tif')
print(dtmTail[0])
generalRasterTools = RM.generalRasterTools(transitFolderPath)
name,ext = ms.splitFilenameAndExtention(dtmTail[0])
mosaikFileNamePermanent = ms.makePath(transitFolderPath,(name + '_mosaik.tif'))
generalRasterTools.computeMosaic(mosaikFileNamePermanent)

In [ ]:
generalRasterTools.rasterToVectorLine('basin4_StrahOrder_level5.tif', 'basin4_SOL5.shp')

In [ ]:
generalRasterTools.rasterVisibility_index('basin4_5m_fill.tif', 'visibilityBasin4.tif')